In [548]:
import tensorflow as tf
import numpy as np

In [549]:
pole = np.sqrt(3) - 2
l = 6
def init_causal_coeff(f):
    # first perform interpolation on the axis 1 (axis 0 is batch axis)
    _, h, _, _ = f.shape
    horizon = min(12, h)  # the minimum iterations is 12
    zk = tf.reshape(tf.range(0, horizon, dtype=f.dtype), (1, horizon, 1, 1))
    zk = tf.pow(pole, zk + 1)
    s = f[:, 0:horizon, ...]

    return tf.reduce_sum(s * zk, axis=1, keepdims=True)

In [550]:
def init_causal_coeff_py(f, data_length):
    horizon = min(12, data_length)
    zk = pole
    s = f[:, 0, ...]
    for k in range(horizon):
        s += zk * f[:, k, ...]
        zk *= pole
    return s

In [551]:
def init_anti_causal_coeff(f):
    return (pole / (pole - 1)) * f[:, -1:, ...]

In [552]:
def cubic_prefilter(f):
    n, h, w, c = f.shape
    cp = tf.concat([init_causal_coeff(f), f[:, 1:, ...]], axis=1) * l

    zp1 = tf.reshape(tf.pow(pole, - tf.range(0, h, dtype=f.dtype)), (1, h, 1, 1))

    cp = tf.cumsum(zp1 * cp, axis=1) / zp1
    cm = tf.concat([cp[:, :-1, ...], -init_anti_causal_coeff(cp) / pole], axis=1)
    zp2 = tf.reshape(tf.pow(pole, tf.range(0, h, dtype=f.dtype) + 1), (1, h, 1, 1))
    cm = tf.cumsum(cm * zp2, reverse=True, axis=1) * (-zp1)
    return cm

In [553]:
def cubic_prefilter_py(f, data_length):
    f[:, 0, ...] = l * init_causal_coeff_py(f, data_length)
    for k in range(1, data_length):
        f[:, k, ...] = l * f[:, k, ...] + pole * f[:, k - 1, ...]
    f[:, data_length - 1, ...] = init_anti_causal_coeff(f)
    # print(f)
    for k in range(data_length - 2, -1, -1):
        f[:, k, ...] = pole * (f[:, k + 1, ...] - f[:, k, ...])
    return f

In [554]:
t = np.reshape(np.linspace(0, 1, 10), (1, 10, 1, 1))

In [555]:
cubic_prefilter_py(t.copy(), 10)

array([[[[-0.02347995]],

        [[ 0.11740312]],

        [[ 0.22053412]],

        [[ 0.33379372]],

        [[ 0.44429098]],

        [[ 0.55570902]],

        [[ 0.66620626]],

        [[ 0.77946593]],

        [[ 0.88259669]],

        [[ 1.02348066]]]])

In [556]:
cubic_prefilter(t.copy())

<tf.Tensor: shape=(1, 10, 1, 1), dtype=float64, numpy=
array([[[[-0.02347995]],

        [[ 0.11740312]],

        [[ 0.22053412]],

        [[ 0.33379372]],

        [[ 0.44429098]],

        [[ 0.55570902]],

        [[ 0.66620626]],

        [[ 0.77946593]],

        [[ 0.88259669]],

        [[ 1.02348066]]]])>

In [ ]:
lam = 6
zp = np.sqrt(3) - 2
def cubic_prefilter(f):
    n, h, w, c = f.shape
    dtype = f.dtype

    zp_fill = tf.fill(shape=(1, h, 1, 1), value=zp, dtype=f.dtype)

    cumprod1 = tf.math.cumprod(zp_fill, axis=1)
    cumprod2 = tf.math.cumprod(zp_fill, axis=1, reverse=True) * tf.math.pow(zp, h)

    cp_bound = lam * (f[:, 0, :, :]) + (1 / (1 - tf.pow(zp, 2 * h))) * tf.reduce_sum((cumprod1 + cumprod2) * f, axis=1)

    cp = tf.math.cumsum(cumprod2[1:] / tf.math.pow(zp, h) * lam * f[:, 1:, :, :], axis=1) * \
         tf.math.pow(zp, tf.reshape(tf.range(h, dtype=dtype)[1:], (1, -1, 1, 1)) - h)

    cp = tf.pad(cp, [(0, 0), (1, 0), (0, 0), (0, 0)], constant_values=cp_bound)

    cm = - tf.cumsum()